In [1]:
import cv2
import numpy as np

# Used for distance calculation later
from sklearn.metrics import pairwise

In [2]:
background = None

accumulated_weight = 0.5

roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [3]:
def calc_accum_avg(frame, accumulated_weight):
    global background
    
    if background is None:
        background = frame.copy().astype('float')
        return None
    
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [4]:
#Segments hand region using thresholds and contours
def segment(frame, threshold_min=25):
    diff = cv2.absdiff(background.astype('uint8'), frame)
    ret,thresholded = cv2.threshold(diff, threshold_min, 255, cv2.THRESH_BINARY)
    
    image,contours,hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    
    else:
        #Assuming largest external contour in ROI is the hand
        hand_segment = max(contours, key=cv2.contourArea)
        
        return (thresholded, hand_segment)

In [5]:
#Estimating the center of the hand
def count_fingers(thresholded, hand_segment):
    conv_hull = cv2.convexHull(hand_segment)
    
    #Most extreme top point
    top = tuple(conv_hull[conv_hull[:,:,1].argmin()][0])
    #Bottom
    bottom = tuple(conv_hull[conv_hull[:,:,1].argmax()][0])
    #Left
    left = tuple(conv_hull[conv_hull[:,:,0].argmin()][0])
    #Right
    right = tuple(conv_hull[conv_hull[:,:,0].argmax()][0])
    
    #Center X and Y
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2
    
    #Distance calculation
    distance = pairwise.euclidean_distances([(cX,cY)], Y=[left,right,top,bottom])[0]
   
    max_distance = distance.max()
    
    #Percentage can be changed
    radius = int(0.8*max_distance)
    
    circumfrence = (2*np.pi*radius)
    
    circular_roi = np.zeros(thresholded.shape[:2], dtype='uint8')
    
    #Draw circle ROI
    cv2.circle(circular_roi, (cX,cY), radius, 255, 10)
    
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)
    
    image,contours,hierarchy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    #Fingers extended
    count = 0
    
    for cnt in contours:
        
        (x,y,w,h) = cv2.boundingRect(cnt)
        
        out_of_wrist = ((cY + (cY*0.25)) > (y+h))
        
        limit_points = ((circumfrence*0.25) > cnt.shape[0])
        
        if out_of_wrist and limit_points:
            count += 1
            
    return count

In [6]:
#Putting it all together
cam = cv2.VideoCapture('http://192.168.0.2:8080/video')

num_frames = 0

#first 60 frames calculate background avg before hand in ROI

while True:
    ret,frame = cam.read()
    
    # flip the frame so that it is not the mirror view
    frame = cv2.flip(frame, 1)
    
    frame_copy = frame.copy()
    
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]
    
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    
    #Blurr
    gray = cv2.GaussianBlur(gray, (7,7), 0)
    
    if num_frames < 60:
        calc_accum_avg(gray, accumulated_weight)
        
        if num_frames <= 59:
            cv2.putText(frame_copy, 'WAIT. Getting Background!', (150,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2)
            cv2.imshow('Finger Count', frame_copy)
    else:
        hand = segment(gray)
        
        if hand is not None:
            thresholded,hand_segment = hand
            
            #Draws hand contours in live stream
            cv2.drawContours(frame_copy, [hand_segment+(roi_right, roi_top)], -1, (255,0,0), 1)
            
            fingers = count_fingers(thresholded, hand_segment)
            
            cv2.putText(frame_copy, str(fingers), (70,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
            cv2.imshow('Thresholded', thresholded)
            
    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)

    num_frames +=1

    cv2.imshow('Finger Count', frame_copy)

    k = cv2.waitKey(1) & 0xFF
    
    if k == 27:
        break
cam.release()
cv2.destroyAllWindows()